# Setup ipynb
IPython notebook that imports everything that we need and provide some functions that we'll use later in other notebooks which I don't want to litter.

In [1]:
# Import warnings first to suppress import logs.
import warnings
warnings.resetwarnings()
warnings.filterwarnings('ignore')

## Imports

In [1]:
import gc
import json
import os
import pickle
import random
import re
import string
from collections import defaultdict, Counter
from itertools import combinations
from urllib.parse import unquote

import numpy as np
from sklearn.metrics.pairwise import cosine_similarity
from sklearn.model_selection import train_test_split
from sklearn.preprocessing import LabelEncoder

import faiss
import matplotlib.pyplot as plt
import pandas as pd
import seaborn as sns
import sentencepiece as spm
from keras import backend as K
from keras.callbacks import EarlyStopping
from keras.layers import (Activation, Dense, Embedding, GlobalAveragePooling1D,
                          Input, Lambda)
from keras.models import Model
from keras.optimizers import Adam
from keras.utils import to_categorical
from tensorflow import keras
from tensorflow.keras.callbacks import ModelCheckpoint
from tqdm import tqdm_notebook

%matplotlib inline

print('Imports done!')

Using TensorFlow backend.


KeyboardInterrupt: 

## Tools

In [1]:
def get_attr(obj, attr):
    '''From data string for a given object retrieves attribute.'''
    return json.loads(obj)[attr]


def get_array_attr(obj, attr, id_='id'):
    '''From data string for a given object retrieves id_ from each item of attr array.'''
    array = []
    
    for piece in json.loads(obj)[attr]:
        if id_ in piece:
            array.append(piece[id_])
    
    return array


def get_artist_of_track(track):
    '''Get artists id of given track'''
    if track in track_to_artist:
        return track_to_artist[track]
    else:
        return -1


def track_array_to_artists(array):
    '''Migrate from array of tracks ids to array of artists ids'''
    artists = list(set(map(get_artist_of_track, array)))
    
    if -1 in artists:
        artists.remove(-1)
    
    return artists


def get_artist_name(id_, df):
    '''Get name of artist with artist_id = id_ from df DataFrame.'''
    data = df[df['artist_id'] == id_]
    
    if len(data) > 0:
        return data.iloc[0]['artist_name'].replace('+', ' ')
    return 'Unknown'

def find_artist(name, df):
    '''In a given Dataframe search all occurrences of artist with a given name.'''
    temp_df = df[df.artist_name.fillna('').str.contains(name)]

    for idx in temp_df['artist_id']:
        try:
            print(df[df.artist_id == idx].
                  iloc[0]['artist_name'].replace('+', ' '), idx)
        except:
            continue
    
    return list(set(temp_df['artist_id']))

def replace_none(array, replacer):
    '''Replace None values in array with replacer value.'''
    
    return [item if item else replacer for item in array]